## MVP submission
Identifying foot traffic for a single Duane Reade store using MTA station entry data.

### Importing and selecting a Duane Reade store

In [117]:
# Import env variable API key
from dotenv import dotenv_values
config = dotenv_values(".env")
API_KEY = config["API_KEY"]

In [159]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

drlocations = pd.read_csv('/Users/joycetagal/GitHub/metis-eda/drlocations_new.csv')

In [142]:
# Code for geocoding latlongs from address. Commented out to ensure API isn't pinged unnecessarily
# import googlemaps
# gmaps = googlemaps.Client(key=API_KEY)
# drlocations["latlon"] = drlocations.address.apply(lambda address : gmaps.geocode(address)[0]['geometry']['location'])
# drlocations = pd.concat([drlocations, drlocations['latlon']
#                         .apply(pd.Series)], axis=1
#                       ).drop("latlon", axis=1)
# drlocations.to_csv("drlocations_new.csv", index=False)

In [161]:
drlocations.head()

,store_name,address,lat,lng
0,Duane Reade,"250 BROADWAY, NEW YORK, NY 10007",40.713008,-74.007821
1,Duane Reade,"305 BROADWAY, NEW YORK, NY 10007",40.715479,-74.005545
2,Duane Reade,"17 JOHN ST NEW YORK, NY 10038",40.709970,-74.008741
3,Duane Reade,"185 GREENWICH ST, NEW YORK, NY 10007",40.711566,-74.011426
4,Duane Reade,"200 WATER ST, NEW YORK, NY 10038",40.707256,-74.004805


For the purposes of this notebook, I will just use the first Duane Reade store in this dataset, i.e. the one at 250 Broadway.

In [162]:
drlocation = drlocations.iloc[[1,]]
drlocation

,store_name,address,lat,lng
1,Duane Reade,"305 BROADWAY, NEW YORK, NY 10007",40.715479,-74.005545


## Identifying isochrones for the DR location

In [156]:
# First we import our MTA stations dataset. For the MVP I'm just using 3 stations.

d = [['City Hall', '40.7131583', '-74.00773'], 
     ['Park Place', '40.7131736', '-74.0092965'], 
     ['Fulton Street', '40.7115643' , '-74.009928']
    ]

In [157]:
stations = pd.DataFrame(d, columns = ['station_name', 'lat', 'lng'])

In [158]:
# Inspiration: https://github.com/ghing/isochrones-python-apis/blob/master/Isochrones%20with%20Python%20and%20APIs.ipynb

stations

,station_name,lat,lng
0,City Hall,40.7131583,-74.00773
1,Park Place,40.7131736,-74.0092965
2,Fulton Street,40.7115643,-74.009928
